In [1]:
import pandas as pd

In [4]:
'''
csvの読み込み
電波途切れた関係で100店舗のもの（../csv/review_100.csv）とそれ以外の全てのデータ(../csv/review_1100.csv)が存在する
これをあとでマージ
'''

review_100 = pd.read_csv('../csv/review_100.csv') # shape(99,8)
review_other = pd.read_csv('../csv/review_1100.csv') # shape(999,8)

In [10]:
review_100['Unnamed: 0']

0      0
1      1
2      2
3      3
4      4
      ..
94    94
95    95
96    96
97    97
98    98
Name: Unnamed: 0, Length: 99, dtype: int64